In [1]:
import os
import pywt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import numpy as np
import random
from pytorch_lightning.loggers import WandbLogger
import wandb
import auraloss
import collections
from tqdm import tqdm
import pretty_midi
import matplotlib.pyplot as plt
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
print(torch.cuda.is_available())
import plotly.graph_objects as go
from torch.optim import lr_scheduler
from demucs import pretrained, htdemucs

True


In [2]:
seed_value = 3407
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.set_float32_matmul_precision('high')

In [3]:
path = "D:/Github/phd-drum-sep/Data/musdb18hq/"

In [4]:
train = list(os.listdir(path+'train'))
test = list(os.listdir(path+'test'))

In [5]:
all_scenes = {}
counter = 0
for idx, val in tqdm(enumerate(train)):
    p = path + 'train/' + val + "/"
    info = torchaudio.info(f"{p}mixture.wav")
    seconds = info.num_frames // 44100
    for i in range(0, seconds - 10, 10):
        start_point = i * 44100
        if start_point + 441000 < info.num_frames:
            all_scenes[counter] = {'music_path': p, 'start_point': start_point, 'length': 441000, 'frames' : info.num_frames}
            counter += 1

100it [00:00, 7364.50it/s]


In [6]:
class AudioDataGenerator(Dataset):
    def __init__(self, data, sample_rate=44100, segment_length = 10, reference="drums.wav"):
        self.data = data
        self.sample_rate = sample_rate
        self.segment_length = sample_rate * segment_length
        self.reference = reference

    def __len__(self):
        return len(self.data)
    
    def load_audio(self, path, start_point, filename):
        audio_tensors = []
        file = filename
        segment, _ = torchaudio.load(f"{path}/{file}", frame_offset=start_point, num_frames=self.segment_length)
        audio_tensors.append(segment)
        return torch.cat(audio_tensors, dim=0)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = self.data[idx]

        # Load audio as a tensor
        audio_path = sample['music_path']

        start_point = sample['start_point']

        mixture_tensor = self.load_audio(audio_path, start_point,'mixture.wav')
        reference_tensor = self.load_audio(audio_path, start_point, self.reference)
        return mixture_tensor, reference_tensor

In [7]:
class AudioDataModule(pl.LightningDataModule):
    def __init__(self, data, batch_size=32, num_workers=0, persistent_workers=False, shuffle=False):
        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.persistent_workers=persistent_workers
        self.shuffle = shuffle

    def setup(self, stage=None):
        # Split your data here if necessary, e.g., into train, validation, test
        self.dataset = AudioDataGenerator(self.data)

    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=self.shuffle, num_workers = self.num_workers, persistent_workers=self.persistent_workers)

    # Implement val_dataloader() and test_dataloader() if you have validation and test data

In [8]:
class Encoder(nn.Module):
    def __init__(self, sample_rate=44100, n_cc=32, hop_length=512, n_fft=2048):
        super(Encoder, self).__init__()
        # Define down-sampling layers for one encoder head
        self.down1 = nn.Sequential(nn.Conv1d(2, 16, kernel_size=15, stride=2, padding=7),  nn.Softsign())
        self.down2 = nn.Sequential(nn.Conv1d(16, 32, kernel_size=15, stride=2, padding=7),  nn.Softsign())
        self.down3 = nn.Sequential(nn.Conv1d(32, 64, kernel_size=13, stride=2, padding=6),  nn.Softsign())
        self.down4 = nn.Sequential(nn.Conv1d(64, 128, kernel_size=11, stride=2, padding=5),  nn.Softsign())
        self.down5 = nn.Sequential(nn.Conv1d(128, 256, kernel_size=9, stride=2, padding=4),  nn.Softsign())
        self.down6 = nn.Sequential(nn.Conv1d(256, 1024, kernel_size=7, stride=2, padding=3),  nn.Softsign())
        self.down7 = nn.Sequential(nn.Conv1d(1024, 1024, kernel_size=5, stride=2, padding=2),  nn.Softsign())
        self.down8 = nn.Sequential(nn.Conv1d(1024, 2048, kernel_size=5, stride=2, padding=2),  nn.Softsign())
        self.down9 = nn.Sequential(nn.Conv1d(2048, 4096, kernel_size=5, stride=2, padding=2),  nn.Softsign())
    
    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)        
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        d9 = self.down9(d8)

        return [d1, d2, d3, d4, d5, d6, d7, d8, d9]  # Return all intermediate outputs for skip connections

In [9]:
class WaveUNet(pl.LightningModule):
    def __init__(self):
        super(WaveUNet, self).__init__()
        # Create 4 separate encoder heads
        self.encoders = Encoder()

        # Define the up-sampling layers
        # Adjust the number of input channels according to the concatenated encoder outputs
        self.up9 = nn.Sequential(nn.ConvTranspose1d(4096, 2048, kernel_size=5, stride=2, padding=2, output_padding=0),  nn.Softsign())
        self.up8 = nn.Sequential(nn.ConvTranspose1d(2048, 1024, kernel_size=5, stride=2, padding=2, output_padding=1),  nn.Softsign())
        self.up7 = nn.Sequential(nn.ConvTranspose1d(1024, 512, kernel_size=5, stride=2, padding=2, output_padding=0), nn.Softsign())
        self.up6 = nn.Sequential(nn.ConvTranspose1d(512, 256, kernel_size=7, stride=2, padding=3, output_padding=1),  nn.Softsign())
        self.up5 = nn.Sequential(nn.ConvTranspose1d(256, 128, kernel_size=9, stride=2, padding=4, output_padding=0),  nn.Softsign())
        self.up4 = nn.Sequential(nn.ConvTranspose1d(128, 64, kernel_size=11, stride=2, padding=5, output_padding=0), nn.Softsign())
        self.up3 = nn.Sequential(nn.ConvTranspose1d(64, 32, kernel_size=13, stride=2, padding=6, output_padding=1), nn.Softsign())
        self.up2 = nn.Sequential(nn.ConvTranspose1d(32, 16, kernel_size=15, stride=2, padding=7, output_padding=1), nn.Softsign())
        self.up1 = nn.Sequential(nn.ConvTranspose1d(16, 2, kernel_size=15, stride=2, padding=7, output_padding=1))
        
         # Additional convolutions for processing concatenated skip connections
        self.conv_skip_8 = nn.Sequential(nn.Conv1d(4096, 2048, kernel_size=5, padding=2), nn.Softsign())  # 1024 (from up3) + 512 (skip)
        self.conv_skip_7 = nn.Sequential(nn.Conv1d(2048, 1024, kernel_size=5, padding=2), nn.Softsign())  # 1024 (from up3) + 512 (skip)
        self.conv_skip_6 = nn.Sequential(nn.Conv1d(1536, 512, kernel_size=5, padding=2), nn.Softsign())  # 1024 (from up3) + 512 (skip)
        self.conv_skip_5 = nn.Sequential(nn.Conv1d(512, 256, kernel_size=5, padding=2), nn.Softsign())  # 512 (from up3) + 256 (skip)
        self.conv_skip_4 = nn.Sequential(nn.Conv1d(256, 128, kernel_size=5, padding=2), nn.Softsign())  # 256 (from up3) + 128 (skip)
        self.conv_skip_3 = nn.Sequential(nn.Conv1d(128, 64, kernel_size=5, padding=2), nn.Softsign())  # 128 (from up3) + 64 (skip)
        self.conv_skip_2 = nn.Sequential(nn.Conv1d(64, 32, kernel_size=5, padding=2), nn.Softsign())  # 64 (from up3) + 32 (skip)
        self.conv_skip_1 = nn.Sequential(nn.Conv1d(32, 16, kernel_size=5, padding=2), nn.Softsign())  # 32 (from up2) + 16 (skip)


        self.loss_fn = auraloss.freq.MultiResolutionSTFTLoss(
                fft_sizes=[1024, 2048, 4096],
                hop_sizes=[256, 512, 1024],
                win_lengths=[1024, 2048, 4096],
                scale="mel", 
                n_bins=160,
                sample_rate=44100,
                device="cuda"
            )

        self.loss_fn_2 = auraloss.time.SISDRLoss()
    
        self.loss_fn_3 = torch.nn.L1Loss()


    def forward(self, audio):
        skip_connections = []
        encoded_output = []

        audio_input = audio
        
        outputs = self.encoders(audio_input)
        encoded_output.append(outputs[-1])  # Use the last output for the main path
        skip_connections.append(outputs[:-1])  # Collect earlier outputs for skip connections

        # Combine the encoded representations with feature embeddings
        combined = torch.cat([o for o in encoded_output], dim=1)

        #8
        u9 = self.up9(combined)

        #8
        skip_8 = torch.cat([u9, skip_connections[0][-1]], dim=1)  
        skip_8 = self.conv_skip_8(skip_8)  # Convolution to adjust channel size
        u8 = self.up8(skip_8)


        #7
        skip_7 = torch.cat([u8, skip_connections[0][-2]], dim=1)  
        skip_7 = self.conv_skip_7(skip_7)  # Convolution to adjust channel size
        u7 = self.up7(skip_7)

        #6
        skip_6 = torch.cat([u7, skip_connections[0][-3]], dim=1)  
        skip_6 = self.conv_skip_6(skip_6)  # Convolution to adjust channel size
        u6 = self.up6(skip_6)

        #5
        skip_5 = torch.cat([u6, skip_connections[0][-4]], dim=1)  
        skip_5 = self.conv_skip_5(skip_5)  # Convolution to adjust channel size
        u5 = self.up5(skip_5)

        #4
        skip_4 = torch.cat([u5, skip_connections[0][-5]], dim=1)  
        skip_4 = self.conv_skip_4(skip_4)  # Convolution to adjust channel size
        u4 = self.up4(skip_4)

        #3
        skip_3 = torch.cat([u4, skip_connections[0][-6]], dim=1)  
        skip_3 = self.conv_skip_3(skip_3)  # Convolution to adjust channel size
        u3 = self.up3(skip_3)

        #2
        skip_2 = torch.cat([u3, skip_connections[0][-7]], dim=1)  
        skip_2 = self.conv_skip_2(skip_2)  # Convolution to adjust channel size
        u2 = self.up2(skip_2)

        #1
        skip_1 = torch.cat([u2, skip_connections[0][-8]], dim=1)  
        skip_1 = self.conv_skip_1(skip_1)  # Convolution to adjust channel size
        u1 = self.up1(skip_1)
        
        # Output
        out = u1
        return out

    def compute_loss(self, outputs, ref_signals):
        loss = self.loss_fn(outputs, ref_signals) + self.loss_fn_2(outputs, ref_signals) +  self.loss_fn_3(outputs, ref_signals)
        return loss
    
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        audio, reference = batch
        
        outputs = self.forward(audio)

        if batch_idx % 512 == 0:
            input_signal = audio[0].cpu().detach().numpy().T
            generated_signal = outputs[0].cpu().detach().numpy().T
            reference_signal = reference[0].cpu().detach().numpy().T 
            wandb.log({'audio_input': [wandb.Audio(input_signal, caption="Input", sample_rate=44100)]})
            wandb.log({'reference_signal': [wandb.Audio(reference_signal, caption="Reference", sample_rate=44100)]})
            wandb.log({'audio_output': [wandb.Audio(generated_signal, caption="Output", sample_rate=44100)]})
             
        loss = self.compute_loss(outputs, reference)         

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    

    def configure_optimizers(self):
        # Define your optimizer and optionally learning rate scheduler here
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
        return [optimizer], [scheduler]
        

In [10]:
model = WaveUNet()
wandb_logger = WandbLogger(project='UnetWithFeats', log_model='all')
audio_data_module = AudioDataModule(all_scenes, batch_size=1, num_workers=0, persistent_workers=False)

In [11]:
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="gpu", 
    devices=-1,
    logger=wandb_logger,
    gradient_clip_val=5,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model, audio_data_module)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hephyrius. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type                    | Params
---------------------------------------------------------
0  | encoders    | Encoder                 | 59.9 M
1  | up9         | Sequential              | 41.9 M
2  | up8         | Sequential              | 10.5 M
3  | up7         | Sequential              | 2.6 M 
4  | up6         | Sequential              | 917 K 
5  | up5         | Sequential              | 295 K 
6  | up4         | Sequential              | 90.2 K
7  | up3         | Sequential              | 26.7 K
8  | up2         | Sequential              | 7.7 K 
9  | up1         | Sequential              | 482   
10 | conv_skip_8 | Sequential              | 41.9 M
11 | conv_skip_7 | Sequential              | 10.5 M
12 | conv_skip_6 | Sequential              | 3.9 M 
13 | conv_skip_5 | Sequential              | 655 K 
14 | conv_skip_4 | Sequential              | 163 K 
15 | conv_skip_3 | Sequential              | 41.0 K
16 | conv_skip_

Training: |                                                                                                   …

OSError: [Errno 28] No space left on device